# How to implement RAG Pattern using Semantic Kernel and Azure Cognitive Search
**NOTE**: This notebook requires that a search index exists with semantic search and vector index enabled.

Follow the steps in the [Notebook](https://github.com/fsaleemm/cognitive-search-vector-pr/blob/main/demo-python/code/azure-search-vector-python-sample.ipynb) to create the vector index with semantic search enabled.

In [1]:
import os

# Create a virtual environment
#os.system('python3 -m venv env')

# Activate the virtual environment
os.system('source env/bin/activate')

# Install dependencies
#os.system('pip install -r requirements.txt')

1

In [ ]:
!python -m pip install python-dotenv==1.0.0
!python -m pip install --upgrade semantic-kernel
!python -m pip install azure-search-documents==11.4.0b9

In [2]:
import os, json

from dotenv import load_dotenv
if not load_dotenv(): raise Exception(".env file not found")

### Setup Semantic Kernel

In [3]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion, AzureTextEmbedding

kernel = Kernel()

kernel.add_text_embedding_generation_service(
    "ada",
    AzureTextEmbedding(
        os.getenv("AZURE_OPENAI_EMBEDDING_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

kernel.add_chat_service(
    "chat",
    AzureChatCompletion(
        os.getenv("AZURE_OPENAI_CHAT_MODEL"),
        os.getenv("AZURE_OPENAI_ENDPOINT"),
        os.getenv("AZURE_OPENAI_API_KEY"),
    ),
)

### Get the intent of the question being asked

In [63]:
#question = "What is the moonthly fare from grand central to fordham?"
#question = "What is the off peak child one way fare from grand central to fordham?"
#question = "What is the total fare from grand central to fordham if traveling peak time as senior citizen?"
#question = "What is the total fare from grand central to fordham if traveling off peak time as disabled person?"
#question = "What is the total fare for ten trips from grand central to fordham if traveling off peak time as disabled person?"
#question = "What is the moonthly fare from fordham to grand central?"

question = "What is the total fare from grand central to fordham if traveling peak time with 1 adult and 2 children?"

In [64]:
#intent detection

from semantic_kernel import PromptTemplate, PromptTemplateConfig, SemanticFunctionConfig


prompt = """Bot: How can I help you?
User: {{$input}}

---------------------------------------------

The intent of the user: """

prompt_config = PromptTemplateConfig(
    description="Gets the intent of the user.",
    type="completion",
    completion=PromptTemplateConfig.CompletionConfig(0.0, 0.0, 0.0, 0.0, 500),
    input=PromptTemplateConfig.InputConfig(
        parameters=[
            PromptTemplateConfig.InputParameter(
                name="input", description="The user's request.", default_value=""
            )
        ]
    ),
)


# Create the SemanticFunctionConfig object
prompt_template = PromptTemplate(
    template=prompt,
    template_engine=kernel.prompt_template_engine,
    prompt_config=prompt_config,
)

function_config = SemanticFunctionConfig(prompt_config, prompt_template)

get_intent = kernel.register_semantic_function(
    skill_name="OrchestratorPlugin",
    function_name="GetIntent",
    function_config=function_config,
)

result_intent = await kernel.run_async(
    get_intent,
    input_str=question,
)

print(result_intent.result)

Asking for the total fare from Grand Central to Fordham during peak time with 1 adult and 2 children.


### Get relevant data from the ACS index using hybrid search

In [65]:
#get relevant data for context
filenames = ['data/zone2.txt', 'data/zone12.txt']

contents = []
for filename in filenames:
    with open(filename, 'r') as f:
        contents.append(f.read().strip())

results = ' '.join(contents)

print(results)

{
    "from-stations": ["Grand Central/Harlem-125th Street"],
    "zoneNumber": "2",
    "to-stations": ["Fordham"],
    "fares": {
        "adult": {
            "peak": {
                "monthly": 199.75,
                "weekly": 71.00,
                "10-trip": 100.00,
                "one-way": 10.00,
                "onboard-fares": 16.00,
                "LIRR-combo": 15.50
            },
            "off-peak": {
                "monthly": 199.75,
                "weekly": 71.00,
                "10-trip": 63.75,
                "one-way": 7.50,
                "onboard-fares": 14.00,
                "LIRR-combo": 15.50
            }
        },
        "senior-Disabled-Medicare": {
            "peak": {
                "10-trip": 100.00,
                "one-way": 10.00
            },
            "offPeak": {
                "10-trip": 50.00,
                "one-way": 5.00
            }
        },
        "child": {
			"peak": {
                "family-fare": 1.00,
         

### Summarize the result

In [66]:
#Summarize

import semantic_kernel


sprompt = """
Considering these facts

Facts: {{$results}}

Question: {{$input}}

Provide a concise answer ('Answer: ') and a separate detailed explanation ('Explanation: '), in two lines.
"""

sprompt_config = PromptTemplateConfig(
    description="Gets the intent of the user.",
    type="completion",
    completion=PromptTemplateConfig.CompletionConfig(0.5, 0.0, 0.0, 0.0, 1024),
    input=PromptTemplateConfig.InputConfig(
        parameters=[
            PromptTemplateConfig.InputParameter(
                name="input", description="The user's request.", default_value=""
            ),
            PromptTemplateConfig.InputParameter(
                name="results", description="The result from grounding data", default_value=""
            )
        ]
    ),
)


# Create the SemanticFunctionConfig object
sprompt_template = PromptTemplate(
    template=sprompt,
    template_engine=kernel.prompt_template_engine,
    prompt_config=sprompt_config,
)

sfunction_config = SemanticFunctionConfig(sprompt_config, sprompt_template)

get_summary = kernel.register_semantic_function(
    skill_name="OrchestratorPlugin",
    function_name="GetSummary",
    function_config=sfunction_config,
)

variables = semantic_kernel.ContextVariables()
variables["input"] = result_intent.result #question #result_intent.result
variables["results"] = results

result_summary = await kernel.run_async(
    get_summary,
    input_vars=variables
)

print(f"Original Question: {question}")
print(f"Optimized Question: {result_intent}")
print("---")
print(result_summary)

Original Question: What is the total fare from grand central to fordham if traveling peak time with 1 adult and 2 children?
Optimized Question: Asking for the total fare from Grand Central to Fordham during peak time with 1 adult and 2 children.
---
Answer: The total fare from Grand Central to Fordham during peak time with 1 adult and 2 children is $21.00.

Explanation: The fare for 1 adult during peak time is $10.00 and the fare for 2 children during peak time is $1.00. Adding these fares together gives a total of $21.00.
